# Consultas MONGODB

Nicolás Fernández y Néstor Villa

In [1]:
# Importamos las clases y el método init_app
from models import Cliente, Producto, Compra, Proveedor, init_app
from bson import ObjectId
from datetime import datetime

# Iniciamos la conexión a la base de datos
init_app()

In [2]:
#1. Listado de todas las compras de un cliente
pipeline = [
    {
        "$group": {
            "_id": "$cliente.nombre",
            "compras": { "$push": { "$toString": "$_id" } }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "compras": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
compras_cliente = list(Compra.aggregate(pipeline, raw=True))

# Itera sobre los resultados
for resultado in compras_cliente:
    print(f"Cliente: {resultado['nombre_cliente']}, Compras: {resultado['compras']}")


Cliente: Carlos López, Compras: ['671e87d26018dfcfed0451fc', '671e87d26018dfcfed045202', '671e87d26018dfcfed045203', '671e87d26018dfcfed045204', '671e87d36018dfcfed04520a', '671e87d36018dfcfed04520b', '671e87d36018dfcfed04520e']
Cliente: Ana Sánchez, Compras: ['671e87d26018dfcfed045207', '671e87d36018dfcfed04520c']
Cliente: Luis Martínez, Compras: ['671e87d26018dfcfed0451fb', '671e87d26018dfcfed045201', '671e87d26018dfcfed045208']
Cliente: María Fernández, Compras: ['671e87d26018dfcfed0451f6', '671e87d26018dfcfed0451f8', '671e87d26018dfcfed0451f9', '671e87d26018dfcfed0451fa', '671e87d26018dfcfed0451fd', '671e87d26018dfcfed0451fe', '671e87d26018dfcfed045205', '671e87d26018dfcfed045206', '671e87d36018dfcfed04520d']
Cliente: Beatriz Gómez, Compras: ['671e87d26018dfcfed0451f7', '671e87d26018dfcfed0451ff', '671e87d26018dfcfed045200', '671e87d36018dfcfed045209', '671e87d36018dfcfed04520f', '671e87d36018dfcfed045210', '671e87d36018dfcfed045211', '671e87d36018dfcfed045212', '671e87d36018dfcfed

In [3]:
# 2. Listado de todos los proveedores para un producto
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$group": {
            "_id": "$nombre",
            "proveedores": { "$push": "$proveedores.nombre" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$_id",
            "proveedores": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
proveedores_producto = list(Producto.aggregate(pipeline, raw=True))

# Itera sobre los resultados
for resultado in proveedores_producto:
    print(f"Producto: {resultado['nombre_producto']}, Proveedores: {resultado['proveedores']}")


Producto: Bufanda de seda, Proveedores: ['Proveedor Este']
Producto: Vestido de fiesta, Proveedores: ['Proveedor Este']
Producto: Gafas de sol, Proveedores: ['Logística Sur']
Producto: Zapatos de vestir, Proveedores: ['Distribuciones Norte']
Producto: Chaqueta de cuero, Proveedores: ['Proveedor Central']
Producto: Camisa de manga larga, Proveedores: ['Modas Paqui']
Producto: Guantes de invierno, Proveedores: ['Proveedor Central']
Producto: Zapatos deportivos, Proveedores: ['Proveedor Este']
Producto: Pantalón vaquero, Proveedores: ['Distribuciones Norte']
Producto: Camiseta de manga corta azul, Proveedores: ['Modas Paqui']
Producto: Camiseta de manga corta roja, Proveedores: ['Modas Paqui']
Producto: Sombrero de paja, Proveedores: ['Proveedor Este']
Producto: Calcetines de algodón, Proveedores: ['Proveedor Este']
Producto: Jersey de lana, Proveedores: ['Proveedor Este']
Producto: Polo de manga corta verde, Proveedores: ['Modas Paqui']
Producto: Traje de baño, Proveedores: ['Logística S

In [4]:
# 3. Listado de todos los productos diferentes comprados por un cliente
pipeline = [
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": "$cliente.nombre",
            "productos": { "$addToSet": "$productos.nombre" }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "productos": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_cliente = list(Compra.aggregate(pipeline, raw=True))

# Muestra los resultados
for resultado in productos_cliente:
    print(f"Cliente: {resultado['nombre_cliente']}, Productos: {resultado['productos']}")

Cliente: Beatriz Gómez, Productos: ['Sombrero de paja', 'Camiseta de manga corta roja', 'Calcetines de algodón', 'Camiseta de manga corta azul', 'Jersey de lana', 'Cinturón de cuero', 'Traje de baño', 'Polo de manga corta verde', 'Camiseta de manga corta negra', 'Bufanda de seda', 'Vestido de fiesta', 'Gafas de sol', 'Chaqueta de cuero', 'Camisa de manga larga', 'Guantes de invierno', 'Zapatos deportivos', 'Pantalón vaquero']
Cliente: Luis Martínez, Productos: ['Camisa de manga larga', 'Chaqueta de cuero', 'Pantalón vaquero', 'Camiseta de manga corta negra', 'Camiseta de manga corta roja', 'Gafas de sol', 'Traje de baño', 'Zapatos de vestir']
Cliente: Carlos López, Productos: ['Camiseta de manga corta blanca', 'Chaqueta de cuero', 'Zapatos deportivos', 'Falda plisada', 'Calcetines de algodón', 'Camiseta de manga corta azul', 'Bufanda de seda', 'Camiseta de manga corta roja', 'Traje de baño', 'Zapatos de vestir', 'Sombrero de paja']
Cliente: Ana Sánchez, Productos: ['Traje de baño', 'Je

In [5]:
# 4. Listado de productos vendidos por “Modas Paqui” cuyo nombre contenga “manga corta”
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$match": {
            "proveedores.nombre": "Modas Paqui",
            "nombre": { "$regex": "manga corta", "$options": "i" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$nombre",
            "proveedor": "$proveedores.nombre"
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_manga_corta = list(Producto.aggregate(pipeline, raw=True))

for producto in productos_manga_corta:
    print(f"Producto: {producto['nombre_producto']}, Proveedor: {producto['proveedor']}")

Producto: Camiseta de manga corta azul, Proveedor: Modas Paqui
Producto: Camiseta de manga corta blanca, Proveedor: Modas Paqui
Producto: Camiseta de manga corta negra, Proveedor: Modas Paqui
Producto: Camiseta de manga corta roja, Proveedor: Modas Paqui
Producto: Polo de manga corta verde, Proveedor: Modas Paqui


In [6]:
# 5. Calcular el peso y volumen total de los productos comprados por un cliente un día determinado
pipeline = [
    {
        "$match": {
            "cliente.nombre": "Beatriz Gómez",
            "fecha_compra": datetime(2024, 4, 11)
        }
    },
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": None,
            "peso_total_kg": { "$sum": "$productos.peso" },
            "volumen_total_metros": {
                "$sum": {
                    "$multiply": [
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.ancho" }, 100] },
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.alto" }, 100] },
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.profundidad" }, 100] }
                    ]
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "peso_total_kg": 1,
            "volumen_total_metros": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
peso_volumen_cliente = list(Compra.aggregate(pipeline, raw=True))

for resultado in peso_volumen_cliente:
    print(f"Peso Total (kg): {resultado['peso_total_kg']}, Volumen Total (m³): {resultado['volumen_total_metros']}")

Peso Total (kg): 106.59, Volumen Total (m³): 0.27638


In [7]:
# 6. Calcular el número medio de envíos por mes y almacén
pipeline = [
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    { "$unwind": "$productos.proveedores.direcciones_almacenes" },
    {
        "$group": {
            "_id": {
                "almacen": "$productos.proveedores.direcciones_almacenes.calle",
                "mes": { "$month": "$fecha_compra" },
                "año": { "$year": "$fecha_compra" }
            },
            "total_envios": { "$sum": 1 }
        }
    },
    {
        "$group": {
            "_id": "$_id.almacen",
            "promedio_envios_mensuales": { "$avg": "$total_envios" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "almacen": "$_id",
            "promedio_envios_mensuales": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
promedio_envios = list(Compra.aggregate(pipeline, raw=True))

for almacen in promedio_envios:
    print(f"Almacén: {almacen['almacen']}, Promedio de Envíos Mensuales: {almacen['promedio_envios_mensuales']}")

Almacén: Avenida de Andalucía, Promedio de Envíos Mensuales: 1.1428571428571428
Almacén: Carrer de la Pau, Promedio de Envíos Mensuales: 2.0
Almacén: Calle de Alcalá, Promedio de Envíos Mensuales: 1.1666666666666667
Almacén: Calle de la Moda, Promedio de Envíos Mensuales: 2.7142857142857144
Almacén: Rua Augusta, Promedio de Envíos Mensuales: 4.0


In [8]:
# 7. Listado con los tres proveedores con más volumen de facturación
pipeline = [
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    {
        "$group": {
            "_id": "$productos.proveedores.nombre",
            "volumen_facturacion": { "$sum": "$precio_compra" }
        }
    },
    { "$sort": { "volumen_facturacion": -1 } },
    { "$limit": 3 },
    {
        "$project": {
            "_id": 0,
            "proveedor": "$_id",
            "volumen_facturacion": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
top_proveedores = list(Compra.aggregate(pipeline, raw=True))

for proveedor in top_proveedores:
    print(f"Proveedor: {proveedor['proveedor']}, Volumen de Facturación: {proveedor['volumen_facturacion']}")

Proveedor: None, Volumen de Facturación: 14796.98
Proveedor: Modas Paqui, Volumen de Facturación: 7822.58
Proveedor: Proveedor Este, Volumen de Facturación: 7280.96


In [9]:
# 8. Listado de almacenes cerca de unas coordenadas determinadas (100km de distancia máxima)
pipeline = [
    {
        "$geoNear": {
            "near": { "type": "Point", "coordinates": [-9.137398, 38.710141] },
            "distanceField": "distancia",
            "maxDistance": 100 * 1000,
            "spherical": True,
            "query": { "direcciones_almacenes.location": { "$exists": True } }
        }
    },
    { "$unwind": "$direcciones_almacenes" },
    { "$sort": { "distancia": 1 } },
    {
        "$project": {
            "_id": 0,
            "nombre_proveedor": "$nombre",
            "direccion_almacen": "$direcciones_almacenes.calle",
            "ciudad": "$direcciones_almacenes.ciudad",
            "distancia": 1,
            "location": "$direcciones_almacenes.location"
        }
    }
]

# Ejecuta el pipeline con raw=True
almacenes_cercanos = list(Proveedor.aggregate(pipeline, raw=True))

for almacen in almacenes_cercanos:
    print(f"Proveedor: {almacen['nombre_proveedor']}, Almacén: {almacen['direccion_almacen']}, "
          f"Ciudad: {almacen['ciudad']}, Distancia: {almacen['distancia']} metros")

Proveedor: Proveedor Este, Almacén: Rua Augusta, Ciudad: Lisboa, Distancia: 0.0 metros


In [10]:
# 9. Listado de compras dentro de un polígono cuyos vértices están definidos
pipeline = [
    {
        "$match": {
            "direccion_envio.location": {
                "$geoWithin": {
                    "$geometry": {
                        "type": "Polygon",
                        "coordinates": [[
                            [-3.8880, 40.5687],
                            [-3.5499, 40.5687],
                            [-3.5499, 40.3120],
                            [-3.8880, 40.3120],
                            [-3.8880, 40.5687]
                        ]]
                    }
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "compra_id": "$_id",
            "cliente": "$cliente.nombre",
            "productos": "$productos.nombre",
            "direccion_envio": "$direccion_envio",
            "location": "$direccion_envio.location"
        }
    }
]

# Ejecuta el pipeline con raw=True
compras_poligono = list(Compra.aggregate(pipeline, raw=True))

for compra in compras_poligono:
    print(f"Compra ID: {compra['compra_id']}, Cliente: {compra['cliente']}, "
          f"Productos: {compra['productos']}, Dirección: {compra['direccion_envio']}")

In [11]:
# 10. Guardar el listado de compras en una nueva colección
pipeline = [
    {
        "$match": {
            "fecha_compra": datetime(2024, 4, 11)
        }
    },
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    { "$unwind": "$productos.proveedores.direcciones_almacenes" },
    {
        "$match": {
            "productos.proveedores.direcciones_almacenes.calle": "Avenida de Andalucía",
            "productos.proveedores.direcciones_almacenes.ciudad": "Sevilla",
            "productos.proveedores.direcciones_almacenes.location.coordinates": [-5.963709, 37.387533]
        }
    },
    {
        "$project": {
            "_id": 1,
            "cliente": "$cliente.nombre",
            "productos": "$productos.nombre",
            "precio_compra": 1,
            "fecha_compra": 1,
            "direccion_envio": 1,
            "almacen": {
                "direccion": "$productos.proveedores.direcciones_almacenes.calle",
                "ciudad": "$productos.proveedores.direcciones_almacenes.ciudad",
                "location": "$productos.proveedores.direcciones_almacenes.location"
            }
        }
    },
    {
        "$out": "compras_a_enviar"
    }
]

Compra.aggregate(pipeline)

compras_a_enviar = list(Compra.db.database["compras_a_enviar"].find())
for compra in compras_a_enviar:
    print(compra)

{'_id': ObjectId('671e87d36018dfcfed045212'), 'precio_compra': 158.92000000000002, 'fecha_compra': datetime.datetime(2024, 4, 11, 0, 0), 'direccion_envio': None, 'cliente': 'Beatriz Gómez', 'productos': 'Gafas de sol', 'almacen': {'direccion': 'Avenida de Andalucía', 'ciudad': 'Sevilla', 'location': {'type': 'Point', 'coordinates': [-5.963709, 37.387533]}}}
{'_id': ObjectId('671e87d36018dfcfed045213'), 'precio_compra': 471.22, 'fecha_compra': datetime.datetime(2024, 4, 11, 0, 0), 'direccion_envio': None, 'cliente': 'Beatriz Gómez', 'productos': 'Traje de baño', 'almacen': {'direccion': 'Avenida de Andalucía', 'ciudad': 'Sevilla', 'location': {'type': 'Point', 'coordinates': [-5.963709, 37.387533]}}}
